# Importando módulos 

In [1]:
import obspy

from multiprocessing import Pool
from obspy import read,UTCDateTime,Trace
from obspy.clients.fdsn import Client
import os
import glob
import numpy as np
from collections import defaultdict
import pandas as pd
from pandas import Timestamp, date_range

from tqdm import tqdm


import itertools

#para plotar as figuras
#import matplotlib.animation as animation
import matplotlib.pyplot as plt
from matplotlib.transforms import offset_copy
import matplotlib.ticker as ticker
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import ListedColormap
import matplotlib.dates as mdates
from mpl_toolkits.axes_grid1.inset_locator import InsetPosition,inset_axes
import matplotlib
from matplotlib.dates import YearLocator, MonthLocator, DayLocator, HourLocator, MinuteLocator, SecondLocator, DateFormatter
from matplotlib.ticker import MultipleLocator, FormatStrFormatter

from shapely.geometry.polygon import LinearRing

import cartopy.io.shapereader as shpreader
import cartopy.crs as ccrs
import cartopy.feature as cfeature

from IPython.display import HTML
from IPython import display

# Inputs e Outputs

In [2]:
FOLDER_OUTPUT = '/media/sysop/8d2362fc-3b46-49a7-a864-19b2a6ad097b/diogoloc/dados_posdoc/gliders_project/OUTPUT/'

MSEED_INPUT = "/media/sysop/8d2362fc-3b46-49a7-a864-19b2a6ad097b/diogoloc/dados_posdoc/gliders_project/OUTPUT/MSEED_LFI/"

EVENTs_DATA_FOLDER = '/home/sysop/Documents/codes_marefone/EVENT_MSEED/'

SELECT_EVENTs_DATA_FOLDER = '/home/sysop/Documents/codes_marefone/EVENT_MSEED_SEL/'

METADATA_OUTPUT = "/media/sysop/8d2362fc-3b46-49a7-a864-19b2a6ad097b/diogoloc/dados_posdoc/gliders_project/gliders_data/info_csv/metadados_glider_acustico_pmpas-bs.csv"

campanha_LFI_csv = '/media/sysop/8d2362fc-3b46-49a7-a864-19b2a6ad097b/diogoloc/dados_posdoc/gliders_project/Equipamentos/informacoes_sensores_lfi/Dados das aquisicoes_LFIs.xls'

campanha_LFI_posicoes_csv = '/media/sysop/8d2362fc-3b46-49a7-a864-19b2a6ad097b/diogoloc/dados_posdoc/gliders_project/Equipamentos/informacoes_sensores_lfi/LFIs_Posicoes_correto.xlsx'

# Extraindo informações dos arquivos ".mseed"

In [3]:
filenames_MSEED = sorted(glob.glob(MSEED_INPUT+'*/*/*.mseed'))

In [4]:
len(filenames_MSEED)

1892563

In [5]:
def mseed_to_df(o):
    k = o.split('/')[-1]

    if len(k.split('.')) == 4:
        starttime = k.split('.')[2] 
     
    if len(k.split('.')) == 3:
        starttime = k.split('.')[1]

    sensor = k.split('.')[0]

    df = pd.DataFrame([[o],[sensor],[starttime]], index=['file', 'sensor', 'starttime']).T

    return df
    

In [6]:
df_lst = []
with Pool(processes=8) as p:
    with tqdm(total=len(filenames_MSEED)) as pbar:
        for result in p.imap_unordered(mseed_to_df,filenames_MSEED):
            pbar.update()
            df_lst.append(result)

100%|███████████████████████████████| 1892563/1892563 [07:07<00:00, 4427.79it/s]


In [7]:
dataframe_files = pd.concat(df_lst, ignore_index=True)
dataframe_files['starttime'] = pd.to_datetime(dataframe_files.starttime, format='%Y%m%dT%H%M%SZ')

In [10]:
dataframe_files.sort_values(by=['starttime'])

,file,sensor,starttime
106,/media/sysop/8d2362fc-3b46-49a7-a864-19b2a6ad0...,AMAR187,2019-12-06 15:00:01
54,/media/sysop/8d2362fc-3b46-49a7-a864-19b2a6ad0...,AMAR169,2019-12-06 15:00:01
3,/media/sysop/8d2362fc-3b46-49a7-a864-19b2a6ad0...,AMAR164,2019-12-06 15:00:01
53,/media/sysop/8d2362fc-3b46-49a7-a864-19b2a6ad0...,AMAR169,2019-12-06 15:10:01
109,/media/sysop/8d2362fc-3b46-49a7-a864-19b2a6ad0...,AMAR187,2019-12-06 15:10:01
...,...,...,...
1892557,/media/sysop/8d2362fc-3b46-49a7-a864-19b2a6ad0...,AMAR164,2022-08-02 00:50:01
1892559,/media/sysop/8d2362fc-3b46-49a7-a864-19b2a6ad0...,AMAR164,2022-08-02 01:00:01
1892560,/media/sysop/8d2362fc-3b46-49a7-a864-19b2a6ad0...,AMAR164,2022-08-02 01:10:01
1892561,/media/sysop/8d2362fc-3b46-49a7-a864-19b2a6ad0...,AMAR164,2022-08-02 01:20:01


In [11]:
dataframe_files.to_feather('/media/sysop/8d2362fc-3b46-49a7-a864-19b2a6ad097b/diogoloc/dados_posdoc/gliders_project/OUTPUT/NETTAB/df_LFI_dataframe_files.feather')

# Dados LFI

Em cada linha, foram instalados gravadores acústicos nas profundidades de 50 m, 200 m e 950 m. 

No primeiro ciclo, foram lançadas seis linhas e no segundo, quatro linhas.

|Fundeio|Profundidade(m)|Região|
|---:|:---------|:---------------------------------|
|FA01|2.200|Impactada por atividade E&P|
|FA02|2.000|Em desenvolvimento de atividade E&P|
|FA03|2.850|Região sem atividade E&P|
|FA04|1.100|Região sem atividade E&P|
|FA05|1.100|Rota de navegação de E&P|
|FA06|1.100|Rota de navegação de E&P|





## LINHA DE FUNDEIO INSTRUMENTADA (LFI)

E.1 DESCRIÇÃO DO AMAR-G3

O equipamento AMAR-G3 (Autonomous Multichannel Acoustic Recorder) é um gravador acústico autônomo fabricado pela empresa JASCO Applied
Sciences.

O equipamento tem as seguintes características:
* Possibilidade de gravação agendada;
* Capacidade de até 1.792 GBytes de gravação;
* Até 8 canais de 24 bits com frequência de digitalização até 128
kHz;
* Um canal de 16 bits de alta velocidade, até 687,5 kHz;
* Profundidade de operação até 250m, no modo Standard e até

2.500m no modo Deep;
* Dimensões de 16,5 cm de diâmetro e 56,2 cm de altura;
* Opções de 9, 48, 96 ou 144 baterias alcalinas tamanho D;
* Tempo de gravação de até um ano;
* Ganho ajustável de O a 42 dB;
* Dez canais para medições oceanográficas.

O gravador acústico é equipado com um hidrofone modelo M14-V20-370 ou M36-V35-100, da GeoSpectrum. 

O modelo M14-V20-370 possui as seguintes características:
* Faixa de frequência: 10 Hz a 32 kHz;
* Pré-amplificação embutida de 20 dB;
* Resposta de frequência plana até 40 kHz (+/- 2 dB).

Já o modelo M36-V35-100 tem as seguintes características:
* Faixa de frequência 25 Hz a 60 kHz;
* Pré-amplificação embutida de 35 dB
* Resposta de frequência plana até 30 kHz (+/- 2 dB).

# Extraindo informações do arquivo ".csv" com as campanhas e as localizações

In [78]:
df_LFI = pd.read_excel(campanha_LFI_csv)
df_LFI_posicoes = pd.read_excel(campanha_LFI_posicoes_csv)

In [79]:
df_LFI.head(5)
df_LFI.sort_values(by='starttime')

,name,ciclo,starttime,endtime,fs,nBits,sensor,preamp,registrador
12,FA05-050m,LFI-C01,2017-09-24 00:00:00,2018-03-27 00:00:00,64000,24,M36-B000905,6,AMAR169
13,FA05-200m,LFI-C01,2017-09-24 00:00:00,2018-03-27 00:00:00,64000,24,M36-B002479,6,AMAR164
14,FA05-950m,LFI-C01,2017-09-24 00:00:00,2018-03-27 00:00:00,64000,24,M36-B000907,6,AMAR187
3,FA02-050m,LFI-C01,2017-10-16 00:00:00,2018-04-12 00:00:00,64000,24,M14-C000807,6,AMAR045
4,FA02-200m,LFI-C01,2017-10-16 00:00:00,2018-04-12 00:00:00,64000,24,M14-C000809,6,AMAR096
...,...,...,...,...,...,...,...,...,...
135,FA02-050m,LFI-C10,2022-08-23 17:00:00,2022-10-12 23:10:00,64000,24,M14-C000803,18,AMAR038
134,FA01-950m,LFI-C10,2022-08-23 20:08:00,2022-09-04 16:00:00,64000,24,M36-F001108,6,AMAR887
137,FA02-950m,LFI-C10,2022-08-24 16:10:00,2022-10-12 23:20:00,64000,24,M36-F000423,6,AMAR117
136,FA02-200m,LFI-C10,2022-08-25 22:26:00,2022-10-12 23:10:00,64000,24,M36-B000907,6,AMAR122


In [80]:
df_LFI_posicoes.head(5)

,Campanha,Linha,Latitude,Longitude
0,1,FA-01,-25.4555,-42.8141
1,1,FA-02,-24.6559,-42.5687
2,1,FA-03,-24.8940,-40.8420
3,1,FA-04,-27.4960,-46.7000
4,1,FA-05,-24.3560,-43.1143


In [81]:
df_LFI['starttime'] = pd.to_datetime(df_LFI.starttime, format='%Y-%m-%d %H:%M:%S').dt.date
df_LFI['endtime'] = pd.to_datetime(df_LFI.endtime, format='%Y-%m-%d %H:%M:%S').dt.date

In [93]:
df_LFI['depth'] = df_LFI['name'].str[5:-1]
df_LFI['name_LFI'] = df_LFI['name'].str[:2]+'-'+df_LFI['name'].str[2:4]
df_LFI['campanha'] = df_LFI['ciclo'].str[5:]
df_LFI["campanha"] = df_LFI["campanha"].astype(int)
df_LFI['sensor_nettab'] = df_LFI['sensor'].str[:3]

In [94]:
df_LFI.sort_values(by='starttime')

,name,ciclo,starttime,endtime,fs,nBits,sensor,preamp,registrador,depth,name_LFI,campanha,latitude,longitude,nettab_name,date_start_nettab,date_end_nettab,sensor_nettab
12,FA05-050m,LFI-C01,2017-09-24,2018-03-27,64000,24,M36-B000905,6,AMAR169,050,FA-05,1,-24.3560,-43.1143,F501A,2017/267,2018/086,M36
13,FA05-200m,LFI-C01,2017-09-24,2018-03-27,64000,24,M36-B002479,6,AMAR164,200,FA-05,1,-24.3560,-43.1143,F501B,2017/267,2018/086,M36
14,FA05-950m,LFI-C01,2017-09-24,2018-03-27,64000,24,M36-B000907,6,AMAR187,950,FA-05,1,-24.3560,-43.1143,F501C,2017/267,2018/086,M36
3,FA02-050m,LFI-C01,2017-10-16,2018-04-12,64000,24,M14-C000807,6,AMAR045,050,FA-02,1,-24.6559,-42.5687,F201A,2017/289,2018/102,M14
4,FA02-200m,LFI-C01,2017-10-16,2018-04-12,64000,24,M14-C000809,6,AMAR096,200,FA-02,1,-24.6559,-42.5687,F201B,2017/289,2018/102,M14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,FA02-050m,LFI-C10,2022-08-23,2022-10-12,64000,24,M14-C000803,18,AMAR038,050,FA-02,10,-24.6821,-42.5587,F20AA,2022/235,2022/285,M14
134,FA01-950m,LFI-C10,2022-08-23,2022-09-04,64000,24,M36-F001108,6,AMAR887,950,FA-01,10,-25.5933,-42.6632,F10AC,2022/235,2022/247,M36
137,FA02-950m,LFI-C10,2022-08-24,2022-10-12,64000,24,M36-F000423,6,AMAR117,950,FA-02,10,-24.6821,-42.5587,F20AC,2022/236,2022/285,M36
136,FA02-200m,LFI-C10,2022-08-25,2022-10-12,64000,24,M36-B000907,6,AMAR122,200,FA-02,10,-24.6821,-42.5587,F20AB,2022/237,2022/285,M36


In [95]:
def get_lat(line):

    df_pos = df_LFI_posicoes[(df_LFI_posicoes['Linha'] == line['name_LFI']) & (df_LFI_posicoes['Campanha'] == line['campanha'])]

    return df_pos['Latitude'].values[0]

#---------------------

def get_lon(line):

    df_pos = df_LFI_posicoes[(df_LFI_posicoes['Linha'] == line['name_LFI']) & (df_LFI_posicoes['Campanha'] == line['campanha'])]

    return df_pos['Longitude'].values[0]

In [96]:
df_LFI['latitude'] = df_LFI.apply(get_lat,axis=1)
df_LFI['longitude'] = df_LFI.apply(get_lon,axis=1)

In [97]:
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))

In [98]:
def convert_36(line):
       
    campx = line['campanha']

    # número total de campanhas (máximo)
    campanha_range = range(500)
    
    # hexatrigesimal two digit number for each quatrimester:
    hexatri_lst = [str(np.base_repr(y, 36)).zfill(2) for y in range(len(campanha_range))]

    if line['depth'] == '050':
        #find the nearest date and append the 2-hexatrigesimal-digit:
        return line['name_LFI'][0]+line['name_LFI'][-1]+hexatri_lst[campanha_range.index(nearest(campanha_range,campx))]+'A'
    
    if line['depth'] == '200':
        #find the nearest date and append the 2-hexatrigesimal-digit:
       return line['name_LFI'][0]+line['name_LFI'][-1]+hexatri_lst[campanha_range.index(nearest(campanha_range,campx))]+'B'

    if line['depth'] == '950':
        #find the nearest date and append the 2-hexatrigesimal-digit:
        return line['name_LFI'][0]+line['name_LFI'][-1]+hexatri_lst[campanha_range.index(nearest(campanha_range,campx))]+'C'


In [99]:
df_LFI.sort_values(by='starttime')

,name,ciclo,starttime,endtime,fs,nBits,sensor,preamp,registrador,depth,name_LFI,campanha,latitude,longitude,nettab_name,date_start_nettab,date_end_nettab,sensor_nettab
12,FA05-050m,LFI-C01,2017-09-24,2018-03-27,64000,24,M36-B000905,6,AMAR169,050,FA-05,1,-24.3560,-43.1143,F501A,2017/267,2018/086,M36
13,FA05-200m,LFI-C01,2017-09-24,2018-03-27,64000,24,M36-B002479,6,AMAR164,200,FA-05,1,-24.3560,-43.1143,F501B,2017/267,2018/086,M36
14,FA05-950m,LFI-C01,2017-09-24,2018-03-27,64000,24,M36-B000907,6,AMAR187,950,FA-05,1,-24.3560,-43.1143,F501C,2017/267,2018/086,M36
3,FA02-050m,LFI-C01,2017-10-16,2018-04-12,64000,24,M14-C000807,6,AMAR045,050,FA-02,1,-24.6559,-42.5687,F201A,2017/289,2018/102,M14
4,FA02-200m,LFI-C01,2017-10-16,2018-04-12,64000,24,M14-C000809,6,AMAR096,200,FA-02,1,-24.6559,-42.5687,F201B,2017/289,2018/102,M14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,FA02-050m,LFI-C10,2022-08-23,2022-10-12,64000,24,M14-C000803,18,AMAR038,050,FA-02,10,-24.6821,-42.5587,F20AA,2022/235,2022/285,M14
134,FA01-950m,LFI-C10,2022-08-23,2022-09-04,64000,24,M36-F001108,6,AMAR887,950,FA-01,10,-25.5933,-42.6632,F10AC,2022/235,2022/247,M36
137,FA02-950m,LFI-C10,2022-08-24,2022-10-12,64000,24,M36-F000423,6,AMAR117,950,FA-02,10,-24.6821,-42.5587,F20AC,2022/236,2022/285,M36
136,FA02-200m,LFI-C10,2022-08-25,2022-10-12,64000,24,M36-B000907,6,AMAR122,200,FA-02,10,-24.6821,-42.5587,F20AB,2022/237,2022/285,M36


In [100]:
df_LFI['nettab_name'] = df_LFI.apply(convert_36,axis=1)

In [101]:
df_LFI['starttime'] = pd.to_datetime(df_LFI.starttime, format='%d-%m-%Y')
df_LFI['endtime'] = pd.to_datetime(df_LFI.endtime, format='%d-%m-%Y')

In [102]:
df_LFI['date_start_nettab'] = df_LFI.starttime.dt.strftime('%Y/%j')
df_LFI['date_end_nettab'] = df_LFI.endtime.dt.strftime('%Y/%j')

In [103]:
df_LFI.to_feather('/media/sysop/8d2362fc-3b46-49a7-a864-19b2a6ad097b/diogoloc/dados_posdoc/gliders_project/OUTPUT/NETTAB/df_LFI_nettab.feather')

In [104]:
df_LFI.sort_values(by='starttime')

,name,ciclo,starttime,endtime,fs,nBits,sensor,preamp,registrador,depth,name_LFI,campanha,latitude,longitude,nettab_name,date_start_nettab,date_end_nettab,sensor_nettab
12,FA05-050m,LFI-C01,2017-09-24,2018-03-27,64000,24,M36-B000905,6,AMAR169,050,FA-05,1,-24.3560,-43.1143,F501A,2017/267,2018/086,M36
13,FA05-200m,LFI-C01,2017-09-24,2018-03-27,64000,24,M36-B002479,6,AMAR164,200,FA-05,1,-24.3560,-43.1143,F501B,2017/267,2018/086,M36
14,FA05-950m,LFI-C01,2017-09-24,2018-03-27,64000,24,M36-B000907,6,AMAR187,950,FA-05,1,-24.3560,-43.1143,F501C,2017/267,2018/086,M36
3,FA02-050m,LFI-C01,2017-10-16,2018-04-12,64000,24,M14-C000807,6,AMAR045,050,FA-02,1,-24.6559,-42.5687,F201A,2017/289,2018/102,M14
4,FA02-200m,LFI-C01,2017-10-16,2018-04-12,64000,24,M14-C000809,6,AMAR096,200,FA-02,1,-24.6559,-42.5687,F201B,2017/289,2018/102,M14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,FA02-050m,LFI-C10,2022-08-23,2022-10-12,64000,24,M14-C000803,18,AMAR038,050,FA-02,10,-24.6821,-42.5587,F20AA,2022/235,2022/285,M14
134,FA01-950m,LFI-C10,2022-08-23,2022-09-04,64000,24,M36-F001108,6,AMAR887,950,FA-01,10,-25.5933,-42.6632,F10AC,2022/235,2022/247,M36
137,FA02-950m,LFI-C10,2022-08-24,2022-10-12,64000,24,M36-F000423,6,AMAR117,950,FA-02,10,-24.6821,-42.5587,F20AC,2022/236,2022/285,M36
136,FA02-200m,LFI-C10,2022-08-25,2022-10-12,64000,24,M36-B000907,6,AMAR122,200,FA-02,10,-24.6821,-42.5587,F20AB,2022/237,2022/285,M36


In [109]:
lines_nettab = []
for i in df_LFI.sort_values(by='starttime').iterrows():
    # Station lines (one line per station/sensor/epoch) # code description datalogger%sn seismometer%sn # sampling orientation lat. lon. elev. depth. start end
    #Sl: UNAP "Uni-Iquique/Chile"   DM24%A1383 CMG-3ESP/60%T34622 100 ZNE -20.24393 -70.14041 0.0    0.0 2009/134 
    print('Sl: '+i[1].nettab_name+' "LFI:'+i[1]['name']+'" AMARG3A '+i[1].sensor_nettab+' 100 H '+"{:.4f}".format(i[1].latitude)+' '+"{:.4f}".format(i[1].longitude)+' '+str(i[1].depth)+' 0.0 '+i[1].date_start_nettab+' '+i[1].date_end_nettab)   
    lines_nettab.append('Sl: '+i[1].nettab_name+' "LFI:'+i[1]['name']+'" AMARG3A '+i[1].sensor_nettab+' 100 H '+"{:.4f}".format(i[1].latitude)+' '+"{:.4f}".format(i[1].longitude)+' '+str(i[1].depth)+' 0.0 '+i[1].date_start_nettab+' '+i[1].date_end_nettab)

Sl: F501A "LFI:FA05-050m" AMARG3A M36 100 H -24.3560 -43.1143 050 0.0 2017/267 2018/086
Sl: F501B "LFI:FA05-200m" AMARG3A M36 100 H -24.3560 -43.1143 200 0.0 2017/267 2018/086
Sl: F501C "LFI:FA05-950m" AMARG3A M36 100 H -24.3560 -43.1143 950 0.0 2017/267 2018/086
Sl: F201A "LFI:FA02-050m" AMARG3A M14 100 H -24.6559 -42.5687 050 0.0 2017/289 2018/102
Sl: F201B "LFI:FA02-200m" AMARG3A M14 100 H -24.6559 -42.5687 200 0.0 2017/289 2018/102
Sl: F201C "LFI:FA02-950m" AMARG3A M36 100 H -24.6559 -42.5687 950 0.0 2017/289 2018/102
Sl: F301A "LFI:FA03-050m" AMARG3A M14 100 H -24.8940 -40.8420 050 0.0 2017/289 2018/102
Sl: F301B "LFI:FA03-200m" AMARG3A M14 100 H -24.8940 -40.8420 200 0.0 2017/289 2018/102
Sl: F301C "LFI:FA03-950m" AMARG3A M14 100 H -24.8940 -40.8420 950 0.0 2017/289 2018/102
Sl: F401A "LFI:FA04-050m" AMARG3A M14 100 H -27.4960 -46.7000 050 0.0 2017/292 2018/112
Sl: F401B "LFI:FA04-200m" AMARG3A M14 100 H -27.4960 -46.7000 200 0.0 2017/292 2018/112
Sl: F401C "LFI:FA04-950m" AMARG3

In [110]:
with open('/media/sysop/8d2362fc-3b46-49a7-a864-19b2a6ad097b/diogoloc/dados_posdoc/gliders_project/OUTPUT/NETTAB/df_LFI_nettab.txt', 'w') as f:
    for line in lines_nettab:
        f.write(line)
        f.write('\n')

# Renomeando os arquivos ".mseed":

In [ ]:
dataframe_files_rows = [row for idx,row in dataframe_files.iterrows()]

In [ ]:
def rename_files_LFI(row):

    df_LFI_info = df_LFI[(df_LFI['starttime'] < row['starttime']) & (df_LFI['endtime'] > row['starttime']) & (df_LFI['registrador'] == row['sensor'])]

    st = obspy.read(row['file'])
    st[0].stats.network = 'LF'
    st[0].stats.station = str(df_LFI_info['nettab_name'].values[0])
    st[0].stats.channel = 'HHH'
    
    #<SDSdir>/Year/NET/STA/CHAN.TYPE/NET.STA.LOC.CHAN.TYPE.YEAR.DAY
    #Saving in SDS structure        
    OUTPUT_STREAM = FOLDER_OUTPUT+'MSEED_LFI_SDS/'+st[0].stats.starttime.strftime("%Y")+'/'+st[0].stats.network+'/'+st[0].stats.station+'/HHH.D/'
    os.makedirs(OUTPUT_STREAM,exist_ok=True)
        
    station_name_file = st[0].stats.network+'.'+st[0].stats.station+'..'+st[0].stats.channel+'.D.'+st[0].stats.starttime.strftime("%Y.%j.%H.%M.%S")+'.mseed'
    if os.path.exists(OUTPUT_STREAM+station_name_file) == False:
        st.write(OUTPUT_STREAM+station_name_file, format='MSEED')    

In [ ]:
with Pool(processes=4) as p:
    with tqdm(total=len(dataframe_files_rows)) as pbar:
        for result in p.imap_unordered(rename_files_LFI,dataframe_files_rows):
            pbar.update()